This notebook show how to get the global and variable attributes and save them into a file that can be read later.     
This is useful when one is working on a machine which has not internet connection to the Vocab Server.

In [4]:
using DIVAnd
using DataStructures

## 1. Define the metadata
It takes the form of a dictionary.

In [5]:
metadata = OrderedDict(
"project" => "SeaDataCloud",
"institution_urn" => "SDN:EDMO::3330",
"production" => "DIVA group",
"Author_e-mail" => ["Name <email@something.com>"],
"source" => "Observational data from SeaDataNet",
"comment" => " ",
"parameter_keyword_urn" => "SDN:P35::WATERTEMP",
"search_keywords_urn" => ["SDN:P02::TEMP"],
"area_keywords_urn" => ["SDN:C19::1_2"],
"product_version" => "1.0",
"product_code" => "something-to-decide",
"bathymetry_source" => "The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003",
"netcdf_standard_name" => "sea_water_temperature",
"netcdf_long_name" => "sea water temperature",
"netcdf_units" => "degrees C",
"abstract" => "...",
"acknowledgement" => "...",
"documentation" => "https://doi.org/doi_of_doc",
"doi" => "...");

You also need to have defined the file name, the variable and the domain. These pieces of informations are used to build the preview URL.

In [15]:
fname = "./test.nc"
varname = "Temperature"
dx, dy = 0.125, 0.125
lonr = -4.0:dx:10.
latr = 48.5:dy:62.;

48.5:0.125:62.0

## 2. Get the attributes from the Vocab server

In [17]:
ncglobalattrib, ncvarattrib = SDNMetadata(metadata,fname,varname,lonr,latr)

(OrderedDict("project"=>"SeaDataCloud","institution"=>"Royal Belgian Institute of Natural Sciences, Operational Directorate Natural Environment, Belgian Marine Data Centre","institution_urn"=>"SDN:EDMO::3330","production"=>"DIVA group","Author_e-mail"=>"Name <email@something.com>","source"=>"Observational data from SeaDataNet","comment"=>" ","parameter_keyword"=>"ITS-90 water temperature","parameter_keyword_urn"=>"SDN:P35::WATERTEMP","search_keywords"=>"Temperature of the water column"…), OrderedDict("units"=>"degrees C","standard_name"=>"sea_water_temperature","long_name"=>"sea water temperature"))

### 3. Save the information in a text file
The keys and values are separated by "|", but another separator can be used.

In [20]:
open("ncglobalattrib.txt", "w") do f
    for (i, j) in ncglobalattrib
        write(f, "$i|$j\n")
    end
end

In [21]:
open("ncvarattrib.txt", "w") do f
    for (i, j) in ncvarattrib
        write(f, "$i|$j\n")
    end
end

## 4. Reading
The files generated in the previous step can be read as follows.

In [26]:
"""
Read the attributes from the text file storing the 
different key-value pairs.
"""
function read_attrib_file(filename::String, colsep::String="|")::OrderedDict
    ncattrib = OrderedDict{String,String}()
    open(filename, "r") do fr
        while !eof(fr)
            line = readline(fr)
            linesplit = split(line,colsep)
            global key
            if length(linesplit) == 2
                key = String(linesplit[1]);
                value = String(linesplit[2])
                ncattrib[key] = value
            elseif length(linesplit) == 1
                ncattrib[key] = ncattrib[key] * linesplit[1]
            else
                @warn("There is a problem with the number of column in the text file")
                @show linesplit
            end
        end
    end
    return ncattrib
end

read_attrib_file

In [27]:
ncglobalattrib_read = read_attrib_file("./ncglobalattrib.txt")

OrderedDict{String,String} with 22 entries:
  "project"               => "SeaDataCloud"
  "institution"           => "Royal Belgian Institute of Natural Sciences, Oper…
  "institution_urn"       => "SDN:EDMO::3330"
  "production"            => "DIVA group"
  "Author_e-mail"         => "Name <email@something.com>"
  "source"                => "Observational data from SeaDataNet"
  "comment"               => " "
  "parameter_keyword"     => "ITS-90 water temperature"
  "parameter_keyword_urn" => "SDN:P35::WATERTEMP"
  "search_keywords"       => "Temperature of the water column"
  "search_keywords_urn"   => "SDN:P02::TEMP"
  "area_keywords"         => "North Sea"
  "area_keywords_urn"     => "SDN:C19::1_2"
  "product_code"          => "something-to-decide"
  "product_version"       => "1.0"
  "bathymetry_source"     => "The GEBCO Digital Atlas published by the British …
  "abstract"              => "..."
  "acknowledgement"       => "..."
  "documentation"         => "https://doi.org/doi_

In [28]:
ncvarattrib_read = read_attrib_file("./ncvarattrib.txt")

OrderedDict{String,String} with 3 entries:
  "units"         => "degrees C"
  "standard_name" => "sea_water_temperature"
  "long_name"     => "sea water temperature"